# Feature engineering

## Setup

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm # DO NOT REMOVE

%matplotlib inline

In [ ]:
PROJECT_PATH = Path.joinpath(Path.cwd(), '..')
DATA_PATH = PROJECT_PATH / "data"
CSV_PATH = DATA_PATH / "gym_members_exercise_tracking.csv"

In [ ]:
df = pd.read_csv(CSV_PATH)

## Feature creation

In [ ]:
# Show missing values
_null = df.isnull().sum()

print(_null[_null > 0] if any(_null) else "No missing values")

## Feature encoding

## Feature selection